In [2]:
"""!conda install -c conda-forge openexr-python -y
!pip install waymo-open-dataset-tf-2-11-0==1.5.2  --no-cache-dir 
!pip install --upgrade google-cloud-storage"""

'!conda install -c conda-forge openexr-python -y\n!pip install waymo-open-dataset-tf-2-11-0==1.5.2  --no-cache-dir \n!pip install --upgrade google-cloud-storage'

In [1]:
# Imports
import os
import tarfile
import numpy as np
import tensorflow as tf
import torch 
import matplotlib.pyplot as plt
import tqdm
import pickle

from waymo_open_dataset.wdl_limited.sim_agents_metrics import metric_features
from waymo_open_dataset.wdl_limited.sim_agents_metrics import metrics

from waymo_open_dataset.protos import scenario_pb2
from waymo_open_dataset.protos import sim_agents_metrics_pb2
from waymo_open_dataset.protos import sim_agents_submission_pb2
from google.protobuf import text_format

from waymo_open_dataset.utils.sim_agents import submission_specs
from waymo_open_dataset.utils.sim_agents import test_utils as sim_agents_test_utils
from waymo_open_dataset.utils.sim_agents import visualizations
from waymo_open_dataset.utils import trajectory_utils

# Set matplotlib to jshtml so animations work with colab.
from matplotlib import rc
rc('animation', html='jshtml')

# deactivating the warnings
import warnings
warnings.filterwarnings('ignore')

from download import download_from_gcs
from prepare_dataset import Prepare_train_dataset, Prepare_validation_dataset, Prepare_test_dataset
from data_preprocess import process_waymo_data_with_scenario_proto
from Mydataloader import create_dataloader



2023-05-31 21:34:36.638578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 21:34:40.966312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-31 21:34:40.966351: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-31 21:34:53.876324: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

# Downloader

In [4]:
# Download samples
download_from_gcs('uncompressed/scenario/validation/validation.tfrecord-00000-of-00150')
download_from_gcs('uncompressed/scenario/training/training.tfrecord-00000-of-01000')
download_from_gcs('uncompressed/scenario/testing/testing.tfrecord-00000-of-00150')

validation.tfrecord-00000-of-00150 already exists in waymo_open_dataset_/validation/validation.tfrecord-00000-of-00150
training.tfrecord-00000-of-01000 already exists in waymo_open_dataset_/training/training.tfrecord-00000-of-01000
testing.tfrecord-00000-of-00150 already exists in waymo_open_dataset_/testing/testing.tfrecord-00000-of-00150


# Preprocess

In [26]:
!python data_preprocess.py

2023-05-28 12:16:55.288877: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 12:16:55.588421: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-28 12:16:55.588462: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-28 12:16:59.106980: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

## Trajectory data 

In [2]:
# load scenarios/processed_scenarios_training/sample_1a8cc570d620bd31.pkl
pickle_file = 'scenarios/processed_scenarios_training/sample_1a8cc570d620bd31.pkl'
processed = pickle.load(open(pickle_file, 'rb'))

print('The shape of the original trajectories is: ', processed['track_infos']['trajs'].shape)
print('The shape of the tracks to predict until current time step is: ', processed['track_infos']['tracks_to_predict_until_current'].shape)
print('The shape of the tracks to predict after current time step is: ', processed['track_infos']['tracks_to_predict_future'].shape)
print('The shape of the track of adv until current time step is: ', processed['track_infos']['track_of_adv_until_current'].shape)
print('The shape of the track of adv after current time step is: ', processed['track_infos']['track_of_adv_future'].shape)
print('The shape of the track of other agents until current time step is: ', processed['track_infos']['tracks_of_other_agents_until_current'].shape)
print('The shape of the track of other agents after current time step is: ', processed['track_infos']['tracks_of_other_agents_future'].shape)

The shape of the original trajectories is:  (105, 91, 10)
The shape of the tracks to predict until current time step is:  (4, 11, 10)
The shape of the tracks to predict after current time step is:  (4, 80, 10)
The shape of the track of adv until current time step is:  (11, 10)
The shape of the track of adv after current time step is:  (80, 10)
The shape of the track of other agents until current time step is:  (104, 11, 10)
The shape of the track of other agents after current time step is:  (104, 80, 10)


## Map data

In [3]:
for key in processed['map_infos'].keys():
    print('The (shape, type) of the map info {} is: '.format(key), (np.array(processed['map_infos'][key]).shape, type(processed['map_infos'][key])))
    if key != 'all_polylines':
        print('The map info inside each {} element are: '.format(key), processed['map_infos'][key][0].keys())

The (shape, type) of the map info lane is:  ((751,), <class 'list'>)
The map info inside each lane element are:  dict_keys(['id', 'speed_limit_mph', 'type', 'interpolating', 'entry_lanes', 'exit_lanes', 'left_boundary', 'right_boundary', 'polyline_index'])
The (shape, type) of the map info road_line is:  ((77,), <class 'list'>)
The map info inside each road_line element are:  dict_keys(['id', 'type', 'polyline_index'])
The (shape, type) of the map info road_edge is:  ((79,), <class 'list'>)
The map info inside each road_edge element are:  dict_keys(['id', 'type', 'polyline_index'])
The (shape, type) of the map info stop_sign is:  ((1,), <class 'list'>)
The map info inside each stop_sign element are:  dict_keys(['id', 'lane_ids', 'position', 'polyline_index'])
The (shape, type) of the map info crosswalk is:  ((12,), <class 'list'>)
The map info inside each crosswalk element are:  dict_keys(['id', 'polyline_index'])
The (shape, type) of the map info speed_bump is:  ((18,), <class 'list'>

# Data loader

In [4]:
# Example usage
train_directory = 'scenarios/processed_scenarios_training'
validation_directory = 'scenarios/processed_scenarios_validation'
batch_size = 1
train_dataloader = create_dataloader(train_directory, batch_size)
validation_dataloader = create_dataloader(validation_directory, batch_size)


# Iterate over the dataloader
for i, batch in enumerate(train_dataloader):
    print('----------------- batch {} -----------------'.format(i))
    for j, item in enumerate(batch):
        trajectories, tracks_until_current, tracks_future, adv_until_current, adv_future, other_agents_until_current, other_agents_future = item
        print('----------------- item {} -----------------'.format(j))
        print('The shape of the original trajectories is:', trajectories.shape)
        print('The shape of the tracks to predict until current time step is:', tracks_until_current.shape)
        print('The shape of the tracks to predict after current time step is:', tracks_future.shape)
        print('The shape of the track of adv until current time step is:', adv_until_current.shape)
        print('The shape of the track of adv after current time step is:', adv_future.shape)
        print('The shape of the track of other agents until current time step is:', other_agents_until_current.shape)
        print('The shape of the track of other agents after current time step is:', other_agents_future.shape)
        print()
    if i > 3:
        break  

----------------- batch 0 -----------------
----------------- item 0 -----------------
The shape of the original trajectories is: torch.Size([80, 91, 10])
The shape of the tracks to predict until current time step is: torch.Size([7, 11, 10])
The shape of the tracks to predict after current time step is: torch.Size([7, 80, 10])
The shape of the track of adv until current time step is: torch.Size([11, 10])
The shape of the track of adv after current time step is: torch.Size([80, 10])
The shape of the track of other agents until current time step is: torch.Size([79, 11, 10])
The shape of the track of other agents after current time step is: torch.Size([79, 80, 10])

----------------- batch 1 -----------------
----------------- item 0 -----------------
The shape of the original trajectories is: torch.Size([38, 91, 10])
The shape of the tracks to predict until current time step is: torch.Size([7, 11, 10])
The shape of the tracks to predict after current time step is: torch.Size([7, 80, 10])

In [5]:
def simulate_with_extrapolation_pytorch(tracks_until_current, print_verbose_comments=False,num_rollouts=submission_specs.N_ROLLOUTS):
    """Simulate the scenario with extrapolation using PyTorch.

    Args:
        tracks_until_current: A numpy array of shape [num_objects, num_steps, 10] representing the tracks until current time step.
        num_rollouts: An integer representing the number of rollouts to simulate.

    Returns:
        A numpy array of shape [num_rollouts, num_objects, num_steps, 4] representing the simulated tracks.
    """
    vprint = print if print_verbose_comments else lambda arg: None
    # We can verify that all of these objects are valid at the last step.
    number_of_valid_objects = int(torch.sum(tracks_until_current[:, -1, -1]))
    vprint(f'The number of valid objects at the last step is:{number_of_valid_objects}')
    total_number_of_objects = tracks_until_current.shape[0]
    vprint(f'Out of:{total_number_of_objects} total number of objects')
    states = torch.stack([tracks_until_current[:, :, 0], tracks_until_current[:, :, 1],
                         tracks_until_current[:, :, 2], tracks_until_current[:, :, 6]], dim=-1)

    num_objects, num_steps, _ = states.shape
    last_velocities = states[:, -1, 3] - states[:, -2, 3]
    last_velocities = states[:, -1, :3] - states[:, -2, :3]
    # We also make the heading constant, so concatenate 0. as angular speed.
    last_velocities = torch.cat(
        [last_velocities, torch.zeros((num_objects, 1))], dim=-1)
    # It can happen that the second to last state of these sim agents might be
    # invalid, so we will set a zero speed for them.
    vprint(f'Is any 2nd to last state invalid: {torch.logical_not(tracks_until_current[:, -1, -1]).any()}')
    vprint(f'This will result in either min or max speed to be really large: {torch.max(torch.abs(last_velocities))}')
    valid_diff = torch.logical_and(tracks_until_current[:, -1, -1],
                                tracks_until_current[:, -2, -1])
    # `last_velocities` shape: (n_objects, 4).
    last_velocities = torch.where(valid_diff[:, None],
                                last_velocities,
                                torch.zeros_like(last_velocities))
    vprint(f'Now this should be back to a normal value: {torch.max(torch.abs(last_velocities))}')

    # Now we carry over a simulation. As we discussed, we actually want 32 parallel
    # simulations, so we make this batched from the very beginning. We add some
    # random noise on top of our actions to make sure the behaviors are different.
    # To properly scale the noise, we get the max velocities (average over all
    # objects, corresponding to axis 0) in each of the dimensions (x/y/z/heading).
    NOISE_SCALE = 0.01
    # `max_action` shape: (4,).
    max_action = torch.max(last_velocities, dim=0)[0]
    # We create `simulated_states` with shape (n_rollouts, n_objects, n_steps, 4).
    simulated_states = states.unsqueeze(0).repeat(num_rollouts, 1, 1, 1)
    simulated_states = simulated_states[:, :, -1, :].unsqueeze(2)
    vprint(f'Shape of simulated_states: {simulated_states.shape}')

    for step in range(submission_specs.N_SIMULATION_STEPS):
        current_state = simulated_states[:, :, -1, :]
        # Random actions, take a normal and normalize by min/max actions
        action_noise = torch.randn_like(current_state) * NOISE_SCALE
        actions_with_noise = last_velocities.unsqueeze(0) + (action_noise * max_action)
        next_state = current_state + actions_with_noise
        simulated_states = torch.cat(
            [simulated_states, next_state[:, :, None, :]], dim=2)
        
    # now remove first state, as it is the same as the last state of the previous
    simulated_states = simulated_states[:, :, 1:, :]
    vprint(f'Final Shape of simulated_states: {simulated_states.shape}')
    
    return simulated_states.numpy()
        
        

In [6]:
simulated_tracks_sample = simulate_with_extrapolation_pytorch(tracks_until_current, print_verbose_comments=True)

The number of valid objects at the last step is:3
Out of:3 total number of objects
Is any 2nd to last state invalid: False
This will result in either min or max speed to be really large: 0.5867919921875
Now this should be back to a normal value: 0.5867919921875
Shape of simulated_states: torch.Size([32, 3, 1, 4])
Final Shape of simulated_states: torch.Size([32, 3, 80, 4])


In [7]:
# simulate with extrapolation for all batches

def simulate_with_extrapolation_pytorch_all_batches(dataloader):
    """Simulate the scenario with extrapolation using PyTorch.

    Args:
        dataloader: A dataloader object.

    Returns:
        A numpy array of shape [num_rollouts, num_objects, num_steps, 4] representing the simulated tracks.
    """
    simulated_tracks = []
    for  batch in dataloader:
        for j, item in enumerate(batch):
            trajectories, tracks_until_current, tracks_future, adv_until_current, adv_future, other_agents_until_current, other_agents_future = item
            simulated_tracks.append(simulate_with_extrapolation_pytorch(tracks_until_current))
    return simulated_tracks

In [8]:
simulated_tracks = simulate_with_extrapolation_pytorch_all_batches(train_dataloader)